<a href="https://colab.research.google.com/github/Wushuyen/RetailCompany/blob/main/Python_Excel_20%E5%B7%AB%E5%8F%94%E6%99%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

連線至雲端

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


載入套件

In [13]:
from pathlib import Path
import pandas as pd
import csv
import os
import openpyxl
from openpyxl import load_workbook, Workbook
from openpyxl.chart import BarChart, Reference

指定路徑並讀取路徑內資料

In [14]:
this_dir = Path('/content/drive/MyDrive/py4ex')
FilePath = f"{this_dir}/data"

Files = os.listdir(FilePath)
print(Files)

['January.csv', 'March.csv', 'February.csv']


如果目的地Excel不存在，就建立新的

In [15]:
if not os.path.exists(f"{this_dir}/SalesReport.xlsx"):
    wb = Workbook()
    wb.save(f"{this_dir}/SalesReport.xlsx")

依序讀取每個CSV檔，並以store為依據分組加總amount後做成長條圖

In [16]:
for file in Files:
    # 讀取CSV檔案
    df = pd.read_csv(f"{FilePath}/{file}")

    # 開啟現有Excel活頁簿
    destination_wb = load_workbook(f"{this_dir}/SalesReport.xlsx")

    # 建立新的工作表，命名為CSV檔名
    sheet_name = file[:-4]
    destination_ws = destination_wb.create_sheet(title=sheet_name)

    # 寫入資料
    # 標題
    destination_ws.append(list(df.columns))
    # 資料列
    for row in df.itertuples(index=False, name=None):
        destination_ws.append(row)

    # 以store為依據分組加總amount，寫入第I、J欄
    if 'store' in df.columns and 'amount' in df.columns:
      # 分組加總
      sum = df.groupby('store')['amount'].sum().reset_index()
      # 寫入標題
      destination_ws.cell(row=1, column=9, value='Store')
      destination_ws.cell(row=1, column=10, value='Total Amount')
      # 寫入分類的總金額
      for i, row in enumerate(sum.itertuples(index=False), start=2):
          destination_ws.cell(row=i, column=9, value=row.store)
          destination_ws.cell(row=i, column=10, value=row.amount)

    # 建立柱狀圖

    # 建立圖表物件
    chart = BarChart()
    chart.type = "col"
    chart.title = f"{file[:-4]}各城市銷售總額"
    chart.x_axis.title = "城市"
    chart.y_axis.title = "銷售總額"

    # J欄：銷售總額、I欄：城市
    data_ref = Reference(destination_ws, min_col=10, min_row=1, max_row=1 + len(sum))
    cats_ref = Reference(destination_ws, min_col=9, min_row=2, max_row=1 + len(sum))

    # 加入資料與分類
    chart.add_data(data_ref, titles_from_data=True)
    chart.set_categories(cats_ref)

    # 將圖表放到 I10
    destination_ws.add_chart(chart, "I10")

    # 儲存檔案
    destination_wb.save(f"{this_dir}/SalesReport.xlsx")


if "Sheet" in destination_wb.sheetnames:
    std = destination_wb["Sheet"]
    destination_wb.remove(std)
# 儲存檔案
destination_wb.save(f"{this_dir}/SalesReport.xlsx")
print("所有檔案匯入完成")

所有檔案匯入完成
